In [ ]:
import numpy as np
import pickle
import cv2
import glob
import time
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
from IPython.display import HTML

%matplotlib inline

Link to Pickle files: https://www.dropbox.com/s/rfl7h55k6bb26xr/car_dataset.p?dl=0

## Import & Visualize

car_dataset = pickle.load(open("car_dataset.p", "rb"))
cars, notcars = np.asarray(car_dataset['car']), np.asarray(car_dataset['noncar'])

plt.figure(figsize=(15,15))
for i, img in enumerate(cars[np.random.randint(len(cars), size=5)]):   
    plt.subplot(1,5,i+1)
    plt.imshow(img)
    plt.axis('off')
plt.show()

plt.figure(figsize=(15,15))
for i, img in enumerate(notcars[np.random.randint(len(cars), size=5)]):   
    plt.subplot(1,5,i+1)
    plt.imshow(img)
    plt.axis('off')
plt.show()

# 1. Train a car classifier 

### HOG Features

from skimage.feature import hog
def get_hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(3, 3), vis=False):
    """Extract HOG features from a channel
    
    skimage.feature.hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(3, 3), 
    block_norm=None, visualize=False, visualise=None, transform_sqrt=False, feature_vector=True, multichannel=None)
    
    out : (n_blocks_row, n_blocks_col, n_cells_row, n_cells_col, n_orient) ndarray
    HOG descriptor for the image. If feature_vector is True, a 1D (flattened) array is returned.

    hog_image : (M, N) ndarray, optional
    A visualisation of the HOG image. Only provided if visualize is True.
    """
    if vis:
        hog_features, hog_image = hog(img, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, visualise=vis)
        plt.figure()
        plt.subplot(121)
        plt.imshow(img, cmap='gray')
        plt.subplot(122)
        plt.imshow(hog_image, cmap='gray')
        plt.show()
        return hog_features
    else:
        return hog(img, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block)
    
def dense_SIFT(image, density=0.05, display=False):
    """Generate dense SIFT descriptors
    Args:
      image: A numpy array representing the image, which is of type np.int in range [0, 255] of size
             (H x W x 3) in RGB order (eg. output of cv2.imread).
      density: Decide the density of the descriptors.

    Returns:
      result: A one level 128-dimension list with the mean description value of total descriptors.
    """
    # Convert into gray pic
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY).astype(np.uint8)
    # Generate a sift to calculate description vector later
    sift = cv2.xfeatures2d.SIFT_create()

    # The step to evenly pick the description point
    # Use the short side len multiply density
    step_size = round(min(image.shape[0], image.shape[1]) * density)

    # Calculate the points with step_size
    kp = [cv2.KeyPoint(x, y, step_size) for y in range(0, gray.shape[0], step_size)
          for x in range(0, gray.shape[1], step_size)]

    # Calculate the description vector for each points
    dense_feat = sift.compute(gray, kp)

    # Without flattening
    # return dense_feat[1]
    
    if display:
        cv2.drawKeypoints(gray, kp, image, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        plt.figure()
        plt.imshow(image[:, :, ::-1])
        plt.show()

    # Flatten the n description vectors, n for the num of description points
    return np.mean(dense_feat[1], axis=0)

# Red Channel
_ = get_hog(cars[100][:,:,0], vis=True)
# Blue Channel
_ = get_hog(notcars[200][:,:,1], vis=True)
# Green Channel
_ = get_hog(cars[300][:,:,2], vis=True)

# Y Channel
_ = get_hog(cv2.cvtColor(cars[100], cv2.COLOR_RGB2YUV)[:,:,0], vis=True)
# U Channel
_ = get_hog(cv2.cvtColor(notcars[200], cv2.COLOR_RGB2YUV)[:,:,1], vis=True)
# V Channel
_ = get_hog(cv2.cvtColor(cars[300], cv2.COLOR_RGB2YUV)[:,:,2], vis=True)

sift_f = dense_SIFT(cv2.imread("bbox-example-image.jpg"), display=True)